🚀 부스팅 알고리즘 심화 비교


부스팅의 기본 원리는 같지만, 

XGBoost, LightGBM, CatBoost는 각각 다른 특징과 기술을 사용하여 성능과 속도를 개선합니다.

---

1. XGBoost (eXtreme Gradient Boosting)

- 특징: Gradient Boosting의 속도와 성능을 개선한 모델로, 가장 널리 사용됩니다.  

        병렬 처리, 규제(Regularization) 기능, 결측치 자체 처리 기능 등 강력한 기능들을 제공합니다.

- 장점: 안정적인 성능과 뛰어난 예측 정확도를 자랑하며, 다양한 환경에서 범용적으로 사용하기 좋습니다.

- 단점: LightGBM에 비해 학습 속도가 상대적으로 느리고, 메모리 사용량이 많을 수 있습니다.

---

2. LightGBM (Light Gradient Boosting Machine)


- 특징: XGBoost의 느린 속도와 메모리 문제를 해결하기 위해 개발되었습니다. 

        '리프 중심 트리 성장(Leaf-wise growth)' 방식을 사용해 더 빠르고 효율적인 학습이 가능합니다.

- 장점: XGBoost보다 훨씬 빠른 학습 속도와 적은 메모리 사용량을 가집니다. 대용량 데이터 처리에서 특히 강력합니다.

- 단점: 과적합(Overfitting)에 취약할 수 있으므로, 하이퍼파라미터 튜닝에 더 신경 써야 합니다.

---

3. CatBoost (Categorical Boosting)


- 특징: 범주형(Categorical) 특성을 자동으로 처리하는 기능이 내장되어 있습니다. 

        전처리 단계에서 One-Hot Encoding 같은 작업이 필요 없어 편리합니다.

- 장점: 범주형 데이터를 많이 포함한 데이터셋에서 강력한 성능을 보입니다. 하이퍼파라미터 튜닝에 대한 의존도가 비교적 낮습니다.

- 단점: 다른 모델에 비해 학습 속도가 느리고, 메모리 사용량이 많을 수 있습니다.

| 항목         | XGBoost           | LightGBM        | CatBoost       |
|--------------|----------------------|---------------|--------------|
| 학습 속도      | 보통               | 매우 빠름        | 느림            |
| 메모리 사용           | 많음           | 적음            | 많음           |
| 범주형 처리     | 수동 (인코딩 필요)  | 수동 (인코딩 필요) |   자동       |
| 과적합 방지        | 좋음 (규제 기능)    | 취약할 수 있음          | 좋음 (균형 잡힌 트리를 만듬)     |



In [4]:
# 부스팅 모델 비교 실습 코드
# 필요한 라이브러리 불러오기
from sklearn.datasets import load_iris              # 붓꽃 데이터셋 로드
from sklearn.model_selection import train_test_split, cross_val_score # 데이터 분할 및 교차 검증
from sklearn.metrics import accuracy_score          # 정확도 평가 함수
import pandas as pd                                 # 데이터 처리 라이브러리
from xgboost import XGBClassifier                   # XGBoost 모델
from lightgbm import LGBMClassifier                 # LightGBM 모델
from catboost import CatBoostClassifier             # CatBoost 모델
from sklearn.preprocessing import StandardScaler    # 파이프라인을 위한 스케일러
from sklearn.pipeline import Pipeline               # 파이프라인 기능

# 1. 붓꽃(iris) 데이터 불러오기 및 분리
iris = load_iris()
X = iris.data                                       # 문제(특성) 데이터
y = iris.target                                     # 정답(레이블) 데이터
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 훈련 및 테스트 데이터로 분할

# 2. 부스팅 모델들 생성
# eval_metric='logloss' 파라미터만 남기고, 더 이상 사용되지 않는 use_label_encoder는 제거
# CatBoost는 'silent=True'와 iterations=100을 추가하여 학습 시간 단축
models = {
    'XGBoost': XGBClassifier(eval_metric='logloss', random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(silent=True, iterations=100, random_state=42)
}

# 3. 각 모델에 파이프라인 적용, 학습 및 성능 평가
results = {}
for name, model in models.items():
    # 파이프라인 생성 (전처리 + 모델)
    # 붓꽃 데이터는 모두 숫자형이므로 StandardScaler만 사용
    pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                                ('classifier', model)])

    # 파이프라인 학습: fit() 한 번으로 전처리 및 모델 학습이 모두 완료
    print(f"--- {name} 파이프라인 학습 중 ---")
    pipeline.fit(X_train, y_train)

    # 예측: predict() 한 번으로 전처리된 데이터에 대한 예측이 수행
    y_pred = pipeline.predict(X_test)

    # 정확도 계산 및 결과 저장
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy
    print(f"{name} 정확도: {accuracy:.4f}\n")

# 4. 결과 출력
print("### 부스팅 모델별 정확도 최종 비교 ###")
for name, accuracy in results.items():
    print(f"- {name}: {accuracy:.4f}")

# 5. 교차 검증을 통한 안정적인 성능 평가 (선택 사항)
print("\n### 교차 검증(Cross-Validation)을 통한 평균 정확도 ###")
for name, model in models.items():
    # 교차 검증에도 파이프라인을 적용하면 코드가 훨씬 간결해집니다.
    pipeline = Pipeline(steps=[('scaler', StandardScaler()),
                                ('classifier', model)])
    
    # cv=5: 5개의 폴드로 나누어 교차 검증 진행
    scores = cross_val_score(pipeline, X, y, cv=5)
    print(f"- {name}: 평균 정확도 {scores.mean():.4f} (표준편차: {scores.std():.4f})")


--- XGBoost 파이프라인 학습 중 ---
XGBoost 정확도: 1.0000

--- LightGBM 파이프라인 학습 중 ---
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 92
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.073920
[LightGBM] [Info] Start training from score -1.123930
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

/home/ccd/miniconda3/envs/mldl_env_250804/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


- XGBoost: 평균 정확도 0.9533 (표준편차: 0.0163)
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000024 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 90
[LightGBM] [Info] Number of data points in the train set: 120, number of used features: 4
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

/home/ccd/miniconda3/envs/mldl_env_250804/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ccd/miniconda3/envs/mldl_env_250804/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ccd/miniconda3/envs/mldl_env_250804/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ccd/miniconda3/envs/mldl_env_250804/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/home/ccd/miniconda3/envs/mldl_env_250804/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning

- CatBoost: 평균 정확도 0.9667 (표준편차: 0.0211)
